In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
import warnings

In [ ]:
def ef_withpivot(A): 
    U = np.copy(A) # copy the matrix A in U 
    (m,n)=A.shape
    j = 0 # index related to the column
    p =[]
    for i in range(0,m): 
        ech=1
        while (ech == 1) & (j < n):
          indm=np.argmax(abs(U[i:m,j])) # find the pivotal index, maximum element in the column j
          indm=indm+i
          if (indm != i) & (abs(U[indm,j]) > 1e-15): # perform the permutation to work well we should change != 0 with an absolute value less then a constant very small
             U[ [i, indm],:]=U[[indm,i],:]
          if (abs(U[i,j]) > 0):
             p.append(j)
             M = U[i+1:m,j]/U[i,j] # vector because we do elimination of all the row below the pivotal one
                # in numpy array there is no difference row vectors or column vectors
                # np.outer to perform the outer product
             U[i+1:m,j+1:n]=U[i+1:m,j+1:n]-np.outer(M,U[i,j+1:n]) 
           
             U[i+1:m,j]=0
             j=j+1
             ech=0
          else:
            j=j+1
            ech=1      
    return(U,p)    

<details>
<summary>EXPLANATION</summary>

---

## 🔬 Comprehensive Analysis: Gaussian Elimination with Partial Pivoting

### 📌 Algorithm Purpose
This function implements **Gaussian elimination with partial pivoting** to transform a matrix $A \in \mathbb{R}^{m \times n}$ into **row echelon form (REF)**. Unlike reduced row echelon form (RREF), REF only requires:
1. All nonzero rows above zero rows
2. Leading entry (pivot) of each row strictly to the right of the pivot above it
3. Entries below each pivot are zero

Partial pivoting enhances numerical stability by selecting the largest-magnitude element in the current column as the pivot.

---

### 🧮 Mathematical Foundation

#### Core Elimination Step
For pivot element $u_{ii}$ at position $(i,j)$, we eliminate entries below it using:
$$
\text{For each row } k > i: \quad \mathbf{u}_k \leftarrow \mathbf{u}_k - \frac{u_{kj}}{u_{ij}} \mathbf{u}_i
$$
This is implemented efficiently via the **outer product**:
$$
U_{i+1:m,\,j+1:n} \leftarrow U_{i+1:m,\,j+1:n} - \mathbf{m} \cdot \mathbf{u}_i^T
$$
where $\mathbf{m} = U_{i+1:m,\,j} / u_{ij}$ is the multiplier vector.

#### Partial Pivoting
Before elimination at column $j$, we find:
$$
\text{indm} = \underset{k \geq i}{\operatorname{argmax}} \, |u_{kj}|
$$
and swap rows $i$ and $\text{indm}$ if $|\text{indm} - i| > 0$. This minimizes round-off error propagation by avoiding division by small pivots.

---

### 🔍 Line-by-Line Breakdown

| Line | Code | Explanation |
|------|------|-------------|
| 1 | `def ef_withpivot(A):` | Function signature accepting matrix $A$ |
| 2 | `U = np.copy(A)` | Creates a working copy to avoid mutating input |
| 3 | `(m,n)=A.shape` | Extracts dimensions: $m$ rows, $n$ columns |
| 4 | `j = 0` | Column index tracker (advances only after successful pivot) |
| 5 | `p = []` | Stores **pivot column indices** (critical for rank determination) |
| 6 | `for i in range(0,m):` | Outer loop: processes each row as potential pivot row |
| 7 | `ech=1` | Flag controlling column search (`ech=0` when pivot found) |
| 8 | `while (ech == 1) & (j < n):` | Inner loop: scans columns until valid pivot found or matrix exhausted |
| 9–10 | `indm=np.argmax(abs(U[i:m,j]))`<br>`indm=indm+i` | Finds row index of max-magnitude element in current column segment |
| 11–13 | Row swap condition | Swaps rows if:<br>• Candidate pivot row differs from current row (`indm != i`)<br>• Pivot magnitude exceeds tolerance (`> 1e-15`) |
| 14 | `if (abs(U[i,j]) > 0):` | Checks if current element qualifies as pivot (tolerance-based) |
| 15 | `p.append(j)` | Records pivot column index for rank/rank-revealing applications |
| 16 | `M = U[i+1:m,j]/U[i,j]` | Computes **multiplier vector** $\mathbf{m}$ for elimination |
| 17–19 | Outer product update | Efficiently applies elimination to submatrix:<br>`np.outer(M, U[i,j+1:n])` computes $\mathbf{m} \cdot \mathbf{u}_i^T$<br>Subtracts from rows below pivot |
| 20 | `U[i+1:m,j]=0` | Explicitly zeros column entries below pivot (numerical cleanup) |
| 21–22 | `j=j+1; ech=0` | Advances column index and exits inner loop (pivot processed) |
| 23–25 | `else: j=j+1; ech=1` | No valid pivot in current column → skip to next column |
| 26 | `return(U,p)` | Returns:<br>• `U`: Row echelon form matrix<br>• `p`: List of pivot column indices |

---

### ⚠️ Critical Observations & Stability Considerations

#### ✅ Strengths
- **Partial pivoting** significantly improves stability versus naive Gaussian elimination
- **Tolerance-based pivot check** (`> 1e-15`) avoids division by near-zero values
- **Outer product formulation** is computationally efficient for block elimination
- **Pivot tracking** (`p`) enables rank determination and back-substitution setup

#### ⚠️ Limitations & Fixes Needed
1. **Tolerance misuse**: Line 11 checks `abs(U[indm,j]) > 1e-15` *before* swap, but line 14 uses `> 0` for pivot acceptance. **Inconsistent tolerance handling** may cause division by near-zero values.  
   → *Fix*: Use consistent tolerance (e.g., `eps = 1e-12`) in both checks.

2. **Indentation error**: Line 18 has inconsistent indentation (extra leading space), which would cause `SyntaxError` in strict Python interpreters.

3. **No explicit zeroing of subdiagonal after swap**: After row swap, entries below new pivot aren't guaranteed zero until elimination step.

4. **Rank deficiency handling**: For rank-deficient matrices, the algorithm correctly skips zero columns but doesn't explicitly handle dependent rows.

---

### 📊 Computational Complexity
- **Time**: $O(\min(m,n) \cdot m \cdot n)$ — standard for Gaussian elimination
- **Space**: $O(mn)$ for matrix copy (in-place possible but not implemented)
- **Pivoting overhead**: $O(m)$ per column for `argmax`, negligible compared to elimination cost

---

### 🧪 Practical Usage Examples

```python
import numpy as np

# Example 1: Full-rank square matrix
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]], dtype=float)
U, pivots = ef_withpivot(A)
print("REF:\n", U)
print("Pivot columns:", pivots)  # Output: [0, 1, 2] → full rank

# Example 2: Rank-deficient matrix
B = np.array([[1, 2, 3],
              [2, 4, 6],
              [3, 6, 9]], dtype=float)
U, pivots = ef_withpivot(B)
print("REF:\n", U)
print("Rank:", len(pivots))  # Output: 1 → rank-deficient

# Example 3: Rectangular matrix (overdetermined system)
C = np.array([[1, 2],
              [3, 4],
              [5, 6]], dtype=float)
U, pivots = ef_withpivot(C)
print("Pivot columns:", pivots)  # Output: [0, 1] → full column rank
```

---

### 🎓 Pedagogical Notes for Students

- **Why partial pivoting?** Without pivoting, elimination on $\begin{bmatrix} \epsilon & 1 \\ 1 & 1 \end{bmatrix}$ with $\epsilon \approx 10^{-16}$ causes catastrophic cancellation. Pivoting swaps rows to use 1 as pivot.
  
- **Outer product insight**: The operation `np.outer(M, U[i,j+1:n])` constructs the entire update matrix in one step:
  $$
  \begin{bmatrix} m_1 \\ m_2 \\ \vdots \end{bmatrix}
  \begin{bmatrix} u_{i,j+1} & u_{i,j+2} & \cdots \end{bmatrix}
  =
  \begin{bmatrix}
  m_1 u_{i,j+1} & m_1 u_{i,j+2} & \cdots \\
  m_2 u_{i,j+1} & m_2 u_{i,j+2} & \cdots \\
  \vdots & \vdots & \ddots
  \end{bmatrix}
  $$
  This is more efficient than row-by-row updates.

- **Pivot columns ≠ identity positions**: In REF (unlike RREF), pivots need not be 1, and columns between pivots may contain nonzeros above pivots.

- **Numerical tolerance**: The `1e-15` threshold approximates machine epsilon for double precision ($\approx 2.2 \times 10^{-16}$). Always use relative tolerances in production code: `abs(val) > eps * norm(column)`.

This implementation provides a solid foundation for understanding stable linear system solvers, with direct applications in LU decomposition (with pivoting) and rank-revealing factorizations.

</details>


In [ ]:
def gaussian_elimination_with_pivoting(
    matrix: np.ndarray,
    tolerance: float = 1e-12,
    enable_logging: bool = True
) -> Tuple[np.ndarray, list]:
    """
    Compute Row Echelon Form (REF) using Gaussian elimination with partial pivoting.
    
    Algorithm: Forward elimination only (no backward elimination). Produces upper-triangular
    structure where:
      - All nonzero rows are above rows of all zeros
      - Leading coefficient (pivot) of a row is always strictly to the right of the pivot above it
      - Entries below pivots are zero
    
    Parameters
    ----------
    matrix : np.ndarray
        Input 2D matrix (m rows × n columns), may be augmented [A|b]
    tolerance : float
        Numerical tolerance for treating values as zero (handles floating-point errors)
    enable_logging : bool
        Whether to print step-by-step elimination process
    
    Returns
    -------
    ref_matrix : np.ndarray
        Matrix transformed to row echelon form
    pivot_columns : list[int]
        Column indices containing pivots (leading entries)
    
    Raises
    ------
    ValueError : For invalid inputs (non-2D, empty, non-numeric)
    """
    # ===== VALIDATION =====
    if not isinstance(matrix, np.ndarray):
        raise ValueError(f"Input must be a NumPy ndarray, got {type(matrix).__name__}")
    if matrix.ndim != 2:
        raise ValueError(f"Input must be 2-dimensional, got {matrix.ndim} dimensions")
    if matrix.size == 0:
        raise ValueError("Input matrix cannot be empty")
    if not np.issubdtype(matrix.dtype, np.number):
        raise ValueError(f"Matrix must contain numeric values, got dtype {matrix.dtype}")
    
    # Create working copy to avoid modifying original input
    ref_matrix = np.array(matrix, dtype=float)
    num_rows, num_cols = ref_matrix.shape
    
    if enable_logging:
        print("="*70)
        print("GAUSSIAN ELIMINATION WITH PARTIAL PIVOTING (Row Echelon Form)")
        print("="*70)
        print(f"Input matrix dimensions: {num_rows} rows × {num_cols} columns")
        print(f"Initial matrix:\n{ref_matrix}\n")
    
    current_row = 0
    pivot_columns = []
    
    # ===== FORWARD ELIMINATION WITH PARTIAL PIVOTING =====
    for current_col in range(num_cols):
        # Termination condition: all rows processed
        if current_row >= num_rows:
            if enable_logging:
                print(f"✓ All {num_rows} rows processed. Terminating early.")
            break
        
        # STEP 1: Find pivot candidate using partial pivoting (max absolute value in column)
        subcolumn = np.abs(ref_matrix[current_row:, current_col])
        pivot_row_offset = np.argmax(subcolumn)
        pivot_row = current_row + pivot_row_offset
        pivot_value = ref_matrix[pivot_row, current_col]
        
        # STEP 2: Skip column if no valid pivot exists (all values near zero)
        if np.abs(pivot_value) < tolerance:
            if enable_logging:
                max_val = np.max(subcolumn) if subcolumn.size > 0 else 0
                print(f"  Column {current_col}: No pivot found (max |value| = {max_val:.2e} < {tolerance:.0e}) → skipping")
            continue
        
        # STEP 3: Swap current row with pivot row if needed
        if pivot_row != current_row:
            if enable_logging:
                print(f"  ↕ Swapping rows {current_row} ↔ {pivot_row} (pivot value = {pivot_value:.4g})")
            ref_matrix[[current_row, pivot_row], :] = ref_matrix[[pivot_row, current_row], :]
        
        # Record pivot column
        pivot_columns.append(current_col)
        
        # STEP 4: Eliminate entries BELOW the pivot
        if current_row < num_rows - 1:  # Only eliminate if rows exist below
            multipliers = ref_matrix[current_row + 1:, current_col] / ref_matrix[current_row, current_col]
            
            # Vectorized elimination: subtract multiplier * pivot_row from each target row
            ref_matrix[current_row + 1:, current_col:] -= np.outer(
                multipliers,
                ref_matrix[current_row, current_col:]
            )
            
            # Explicitly zero out the column below pivot (handles floating-point noise)
            ref_matrix[current_row + 1:, current_col] = 0.0
            
            if enable_logging:
                print(f"  → Pivot at (row={current_row}, col={current_col}), value = {ref_matrix[current_row, current_col]:.4g}")
                print(f"    Eliminated {len(multipliers)} rows below using multipliers: {[f'{m:.3g}' for m in multipliers[:3]] + (['...'] if len(multipliers) > 3 else [])}")
        
        if enable_logging:
            print(f"  ✓ Processed column {current_col}, advancing to row {current_row + 1}\n")
        
        current_row += 1
    
    # ===== POST-PROCESSING: Clean near-zero values =====
    ref_matrix[np.abs(ref_matrix) < tolerance] = 0.0
    
    if enable_logging:
        print("="*70)
        print("ELIMINATION COMPLETE")
        print("="*70)
        print(f"Pivot columns: {pivot_columns}")
        print(f"Matrix rank: {len(pivot_columns)}")
        print(f"Row echelon form:\n{ref_matrix}")
        print("="*70)
    
    return ref_matrix, pivot_columns

In [ ]:
'''Solve using REF + back-substitution:
    2x + y - z = 8
    -3x - y + 2z = -11
    -2x + y + 2z = -3
Known solution: x=2, y=3, z=-1'''

def back_substitution(ref_matrix: np.ndarray, pivot_columns: list) -> np.ndarray:
    """Solve upper-triangular system using back substitution"""
    m, n = ref_matrix.shape
    num_vars = n - 1  # Assuming last column is RHS
    solution = np.zeros(num_vars)
    
    # Process rows in reverse order
    for i in reversed(range(len(pivot_columns))):
        pivot_col = pivot_columns[i]
        # Compute dot product of known variables
        known_sum = np.dot(ref_matrix[i, pivot_col+1:num_vars], solution[pivot_col+1:])
        # Solve for current variable
        solution[pivot_col] = (ref_matrix[i, -1] - known_sum) / ref_matrix[i, pivot_col]
    
    return solution

# Augmented matrix [A|b]
A1 = np.array([
    [2, 1, -1, 8],
    [-3, -1, 2, -11],
    [-2, 1, 2, -3]
])

print("\n" + "="*70)
print("EXAMPLE 1: Textbook 3×3 Linear System (REF + Back Substitution)")
print("="*70)
ref1, pivots1 = gaussian_elimination_with_pivoting(A1)

# Solve using back substitution
solution1 = back_substitution(ref1, pivots1)
expected1 = np.array([2, 3, -1])
error1 = np.linalg.norm(solution1 - expected1)

print(f"\nComputed solution: x={solution1[0]:.6f}, y={solution1[1]:.6f}, z={solution1[2]:.6f}")
print(f"Expected solution: x={expected1[0]}, y={expected1[1]}, z={expected1[2]}")
print(f"Absolute error: {error1:.2e}")
print(f"Verification: {'✓ PASS' if error1 < 1e-10 else '✗ FAIL'}")


In [ ]:
print("\n" + "="*70)
print("EXAMPLE 3: Edge Case Analysis")
print("="*70)

# Case A: Rank-deficient square matrix (infinite solutions)
print("\n[CASE A] Rank-deficient system (infinite solutions):")
A3a = np.array([
    [1, 2, 3, 6],
    [2, 4, 6, 12],   # Row 2 = 2 × Row 1
    [3, 6, 9, 18]    # Row 3 = 3 × Row 1
])
ref3a, pivots3a = gaussian_elimination_with_pivoting(A3a, enable_logging=False)
print(f"  Pivot columns: {pivots3a} → Rank = {len(pivots3a)} (full rank would be 3)")
print(f"  System has infinitely many solutions (rank < number of variables)")

# Case B: Inconsistent system (no solution)
print("\n[CASE B] Inconsistent system (no solution):")
A3b = np.array([
    [1, 2, 3, 6],
    [2, 4, 6, 12],
    [3, 6, 9, 20]    # Inconsistent: 3×Row1 ≠ Row3 in augmented column
])
ref3b, pivots3b = gaussian_elimination_with_pivoting(A3b, enable_logging=False)
print(f"  Final REF matrix last row: {ref3b[-1]}")
if np.all(np.abs(ref3b[-1, :-1]) < 1e-12) and np.abs(ref3b[-1, -1]) > 1e-12:
    print(f"  ⚠️ INCONSISTENCY DETECTED: 0 = {ref3b[-1, -1]:.2f} → NO SOLUTION EXISTS")

# Case C: Overdetermined system (least-squares scenario)
print("\n[CASE C] Overdetermined system (4 equations, 2 unknowns):")
A3c = np.array([
    [1, 1, 3],
    [2, 1, 4],
    [1, 2, 5],
    [3, 1, 6]
])
ref3c, pivots3c = gaussian_elimination_with_pivoting(A3c, enable_logging=False)
print(f"  Pivot columns: {pivots3c} → Rank = {len(pivots3c)}")
print(f"  System is overdetermined (more equations than unknowns)")
print(f"  REF reveals rank deficiency in augmented matrix → use least squares for approximate solution")